In [3]:
from scipy.optimize import linear_sum_assignment
import numpy as np
import itertools
import networkx as nx
import pandas as pd
import functools
import sys
import cvxpy as cp

# Считывание графа
Т.к. используется nx.Graph, то мультиребер возникнуть не может

In [4]:
data = pd.read_excel('globalterrorismdb_0718dist.xlsx')

In [5]:
def fill_graph(graph, year):
    for row in data.itertuples():
        if row.iyear == year:
            graph.add_node(row.eventid)
            
    for row in data.itertuples():
        if row.iyear == year:
            if pd.notna(row.related):
                for rel in row.related.split(','):
                    rel_int = int(rel)
                    if rel_int in graph:
                        graph.add_edge(row.eventid, rel_int)

In [6]:
G_2001 = nx.Graph()
G_2004 = nx.Graph()

fill_graph(G_2001, 2001)
fill_graph(G_2004, 2004)

G_2001.remove_edges_from(G_2001.selfloop_edges())
G_2004.remove_edges_from(G_2004.selfloop_edges())


G_2001.remove_nodes_from(list(nx.isolates(G_2001)))
G_2004.remove_nodes_from(list(nx.isolates(G_2004)))

# Graph Edit Distance
Воспользуемся сведением задачи подсчета Graph Edit Distance к Linear Sum Assignment Problem, ссылка на статью https://bougleux.users.greyc.fr/articles/ssspr14-approxged.pdf

In [28]:
class NodeGraph():
    def __init__(self, nodes):
        self._nodes = nodes
        
    def nodes(self):
        return self._nodes
    
    def __len__(self):
        return len(self._nodes)

def distance(G_1, G_2, substitution_f=lambda meta_x, x, meta_y, y : 0):
    n, m = len(G_1), len(G_2)
    C = np.zeros((n + m, n + m))
    nodes1 = list(G_1.nodes())
    nodes2 = list(G_2.nodes())

    for i, j in itertools.product(range(n), range(m)):
        C[i, j] = substitution_f(G_1, nodes1[i], G_2, nodes2[j])

    for i, j in itertools.product(range(m), range(m)):
        C[n + i, j] = (1 if i == j else sys.maxsize)

    for i, j in itertools.product(range(n), range(n)):
        C[j, m + i] = (1 if i == j else sys.maxsize)
    
    i_s, j_s = linear_sum_assignment(C)
    return sum(C[i][j] for i, j in zip(i_s, j_s))

def graph_substitution_f(G_1, node_1, G_2, node_2):
    edges_1, edges_2 = list(G_1.edges(node_1)), list(G_2.edges(node_2))

    if len(edges_1) == 0 or len(edges_2) == 0:
        return max(len(edges_1), len(edges_2))
    
    return distance(NodeGraph(edges_1), NodeGraph(edges_2))
    
def graph_edit_distance(G_1, G_2):
    return distance(G_1, G_2, graph_substitution_f)

Проверяем:

In [29]:
graph_edit_distance(G_2004, G_2004)

0.0

In [30]:
graph_edit_distance(G_2001, G_2004)

124.0

Достаточно ничего не говорящее число, чтобы оно было немного понятнее, поделим на суммарные размеры графов(тогда разница между пустым и каким-то графом будет 1, а между одинаковыми - 0)

In [9]:
def normed_ged(G_1, G_2):
    s = G_1.number_of_nodes() + G_2.number_of_nodes()
    return graph_edit_distance(G_1, G_2) / s if s else 0

In [10]:
normed_ged(G_2001, nx.Graph())

1.0

In [11]:
normed_ged(G_2001, G_2004)

0.29314420803782504

Я считаю, что это расстояние очень полезно, в отличие от сравнений для двух графов метрик из следующих заданий.

# Диаметр
Возмем максимальное конечное число в матрице дистанций после выполнения алгоритма Флойда-Уоршелла 

In [21]:
def f_w(G):
    dists = nx.convert_matrix.to_numpy_array(G)
    dists[dists == 0] = np.nan
    np.fill_diagonal(dists, 0)
    n = dists.shape[0]
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if not (np.isnan(dists[i][k]) or np.isnan(dists[k][j])):
                    dists[i][j] = min(dists[i][j], dists[i][k] + dists[k][j])
    return dists

In [22]:
G_2001_dists = f_w(G_2001)
G_2004_dists = f_w(G_2004)

In [23]:
G_2001_dists

array([[ 0.,  1., nan, ..., nan, nan, nan],
       [ 1.,  0., nan, ..., nan, nan, nan],
       [nan, nan,  0., ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ...,  0.,  1.,  1.],
       [nan, nan, nan, ...,  1.,  0.,  1.],
       [nan, nan, nan, ...,  1.,  1.,  0.]])

In [15]:
G_2004_dists

array([[ 0.,  1., nan, ..., nan, nan, nan],
       [ 1.,  0., nan, ..., nan, nan, nan],
       [nan, nan,  0., ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ...,  0.,  1.,  1.],
       [nan, nan, nan, ...,  1.,  0.,  1.],
       [nan, nan, nan, ...,  1.,  1.,  0.]])

Для наглядности.

Диаметры:

In [24]:
np.nanmax(G_2001_dists)

1.0

In [25]:
np.nanmax(G_2004_dists)

1.0

Диаметры получились единичными, и неудивительно, ведь у нас специфические данные - связанные терракты, и я не могу представить, как терракты могут быть связаны "через 1"(или больше), ведь если терракты связаны, то связаны все с друг другом, т.е. у нас должен получаться набор небольших несвязных компонент, каждая из которых представляет собою полносвязный граф(в которых диаметры 1).

Полезной для нас информации этот критерий не дает(учитывая, что о структуре графов мы уже знали).

# Эксцентриситет
Эксцентриситет - функция вершины графа, непонятно что имеется в виду под эксцентриситетом графа, посмотрим на эксцентриситетом всех вершин

In [18]:
np.nanmax(G_2001_dists, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [19]:
np.nanmax(G_2004_dists, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Ну да, получили по единичке на каждую вершину, т.к. изолированные вершины мы удалили, а про единички я выше объясняла.

Ничего полезного в подсчете данной характеристики нет.

# Переферийные вершины
Из предыдущих наблюдений явно следует, что все вершины являются переферийными, посчитать их можно, если бы наши графы не были устроены так просто, вызвав:

In [20]:
(np.nanmax(G_2001_dists, axis=1) == np.nanmax(G_2001_dists)).sum()

271

In [21]:
(np.nanmax(G_2004_dists, axis=1) == np.nanmax(G_2004_dists)).sum()

152

Никакой существенной информации о расстоянии между графами эта метрика не дает, ну кроме того, что в одном в 2 раза больше вершин, чем в другом. По сути, мы изощренным способом посчитали количество вершин графов.

# Girth

По описанию в википедии, это размер наименьшего цикла(т.е. минимум для этого значения - 3). Мы уже знаем, что у нас полные подграфы, т.е. если все компоненты связности пары (пары вершин соединенных одним ребром), то тогда Girth неопределен, т.к. нет циклов, но если есть компонента размера хотя бы 3, то Girth=3, найдем, так ли это:

In [22]:
for node in G_2001:
    neighbours = list(G_2001[node])
    if len(neighbours) > 1:
        print(
            'Вершина {} связана с {}; значит girth=3'.format(
                node,
                ', '.join(map(str, neighbours)),
            )
        )
        break

Вершина 200101150002 связана с 200101150003, 200101150004, 200101150005; значит girth=3


В общем случае нужно было бы написать DFS, но мы же типо на 4 курсе и вы поверите нам что мы умеем его писать.

Т.к. для всех графов из нашего датасета этот параметр будет одинаков, то это абсолютно неинформативно.

# Wiener index
Просто сложим все элементы матрицы расстояний, не забыв, что каждый путь мы учитываем в сумме 2 раза(как i -> j и как j -> i).

In [26]:
np.nansum(G_2001_dists) / 2

322.0

In [27]:
np.nansum(G_2004_dists) / 2

185.0

В принципе, передает некоторую важную информацию о структуре графа, можно использовать.

# Hosoya index

Конечно, можно схитрить и просто перемножить Hosoya index для каждой из наших компонент связности, ведь они являются полными подграфами, то значения для них можно посмотреть хоть в табличке, но нужны баллы, воспользуемся реккурентной: $Z(G) = Z(G-e) + Z(G-\{w,v\})$

И тем фактом, что $Z(G)$ можно посчитать как произведение Z по всем компонентам связности G.

http://evlm.stuba.sk/APLIMAT2018/proceedings/Papers/0933_Seibert_Zahradka.pdf

In [25]:
def hosoya(G):
    copy_wo_e_G = G.copy()
    copy_wo_wv_G = G.copy()
    if G.number_of_edges() > 0:
        edge = list(G.edges)[0]
        copy_wo_e_G.remove_edge(*edge)
        copy_wo_wv_G.remove_nodes_from(edge)
        return hosoya(copy_wo_e_G) + hosoya(copy_wo_wv_G)
    return 1

Проверим корректность по последовательности из википедии https://en.wikipedia.org/wiki/Hosoya_index

Т.е. $hosoya(K_0) = 1, hosoya(K_1) = 1, hosoya(K_2) = 2, hosoya(K_3) = 4, hosoya(K_4) = 10, hosoya(K_5) = 26, \dots$

In [26]:
temp = list(enumerate(nx.connected_components(G_2001)))
temp

[(0, {200101070004, 200101070009}),
 (1, {200101130003, 200101140001}),
 (2, {200101150002, 200101150003, 200101150004, 200101150005}),
 (3, {200101160003, 200101160004, 200101160005}),
 (4, {200101190002, 200101190003, 200101190004, 200101190005}),
 (5, {200101190006, 200101190008, 200101190009, 200101190010}),
 (6, {200101240002, 200101240003}),
 (7, {200101280004, 200101280005, 200101280006}),
 (8, {200101300004, 200101300005}),
 (9, {200102000001, 200102120002}),
 (10, {200102010001, 200102010005, 200102010006}),
 (11, {200102070005, 200102070010}),
 (12, {200102140001, 200102170001}),
 (13, {200102210003, 200102210004}),
 (14, {200103020009, 200103020011}),
 (15, {200103080001, 200103080002, 200103080005}),
 (16, {200103090001, 200103090010}),
 (17, {200103090005, 200103090006}),
 (18, {200103240003, 200103240004, 200103240005}),
 (19, {200103250004, 200103250008}),
 (20, {200104140005, 200104140008}),
 (21, {200104230005, 200104230009}),
 (22, {200105210006, 200105210007}),
 (23,

Второй сабсет иммет размер 4

In [27]:
print(temp[2])
hosoya(G_2001.subgraph(list(nx.connected_components(G_2001))[2]))

(2, {200101150002, 200101150003, 200101150004, 200101150005})


10

30-ый размера 5

In [28]:
print(temp[30])
hosoya(G_2001.subgraph(list(nx.connected_components(G_2001))[30]))

(30, {200107230003, 200107230005, 200107230006, 200107230007, 200107230008})


26

Теперь наша функция:

In [29]:
def count_hosoya_for_subgraphs(G):
    results = [hosoya(G.subgraph(cc)) for cc in nx.connected_components(G)]
    return results

In [30]:
print(count_hosoya_for_subgraphs(G_2001))

[2, 2, 10, 4, 10, 10, 2, 4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 26, 2, 2, 10, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 10, 2, 4, 26, 2, 2, 2, 232, 2, 2, 4, 2, 2, 4, 4, 2, 2620, 2, 2, 4, 2, 2, 2, 2, 4, 4, 10, 2, 2, 4, 4, 10, 10, 764, 4, 3, 4, 2, 4, 2, 2, 2, 2, 4, 76, 2, 2, 2, 2, 76, 2, 26]


Снова убеждаемся, что получающиеся числа из последовательности с википедии. Финальная функция:

In [31]:
Z = lambda G: functools.reduce(lambda x, y : x*y, count_hosoya_for_subgraphs(G))
Z(G_2001)

286860444500050491486623841544050172691006619648000000000

In [32]:
Z(G_2004)

110894011367245689096746565632000

Выглядит очень неинформативно.

## Ядро

Структура ядра, когда мы уже все знаем о наших данных, становится понятной - это по одной вершине из каждой компоненты связности: такое множество будет независимым, причем максимального размера, т.к. из каждой компоненты связности в независимое множество мы можем взять по одной вершине, и не больше, т.к. компоненты - полносвязные графы; это максимальное независимое множество будет также являться доминирующим, т.к. каждая вершина попадает в какую-то из компонент связности, а в каждой компоненте есть вершина, попавшая в независимое множество, а компоненты полносвязны. Если мы возмем по одной вершине из каждой компоненты связности, никакой новой информации о схожести графов мы не получим.

Чтобы решить в общем случае, можно написать линеную программу для задачи целочислного программирования.

Воспользуемся сведением, описанным по ссылке https://studopedia.ru/9_101478_vneshne-ustoychivie-mnozhestva-vershin-grafa.html

С вашим cvxpy у меня как-о не сложилось, поэтому будем использовать ortools.

In [33]:
from ortools.linear_solver import pywraplp

In [34]:
def find_graph_kernel(G):
    order = list(G.nodes())
    n_nodes = len(order)
    G_np = nx.convert_matrix.to_numpy_array(G, order) + np.diag(np.ones(len(order)))
    
    solver = pywraplp.Solver('find_kernel', pywraplp.Solver.BOP_INTEGER_PROGRAMMING)
    objective = solver.Objective()
    
    bool_vars = []
    for node in order:
        bool_vars.append(solver.BoolVar(str(node)))
        objective.SetCoefficient(bool_vars[-1], 1)
    
    for i in range(n_nodes):
        solver.Add(solver.Sum([bool_vars[j] for j in range(n_nodes) if G_np[i][j] == 1]) >= 1)
    
    objective.SetMinimization()
    solver.Solve()

    graph_kernel = []
    for node in bool_vars:
        if node.solution_value() == 1:
            graph_kernel.append(node.name())
     
    return graph_kernel

In [35]:
G_2001_kernel = find_graph_kernel(G_2001)
print(G_2001_kernel)

['200101070009', '200101140001', '200101150004', '200101160005', '200101190004', '200101190009', '200101240003', '200101280006', '200101300005', '200102010006', '200102070010', '200102120002', '200102170001', '200102210004', '200103020011', '200103080005', '200103090006', '200103090010', '200103240005', '200103250008', '200104140008', '200104230009', '200105210007', '200105240002', '200105280004', '200105290006', '200106150002', '200106280002', '200107040004', '200107070008', '200107230005', '200107270005', '200108040002', '200108050004', '200108050020', '200108060008', '200108070004', '200108080006', '200108090012', '200108100002', '200108100009', '200108100012', '200108110002', '200108120006', '200108120012', '200108120017', '200108130011', '200108130015', '200108140017', '200108140024', '200108150002', '200108150008', '200108150015', '200108160007', '200108160020', '200108170008', '200108170014', '200108180002', '200108200016', '200108210013', '200108220004', '200108220005', '200108

In [36]:
print(len(G_2001_kernel))

98


Подтверждение гипотезы, что нужно взять по вершине из связной компоненты:

In [37]:
len(list(nx.connected_components(G_2001)))

98

In [38]:
G_2004_kernel = find_graph_kernel(G_2004)
print(G_2004_kernel)

['200402010009', '200402110004', '200402160009', '200402170002', '200403010002', '200403020003', '200403110003', '200403180004', '200404090003', '200404190004', '200404200004', '200405170002', '200405200002', '200406150006', '200406160012', '200406170002', '200406240004', '200406260003', '200407030004', '200407110006', '200407300003', '200407310008', '200408010002', '200408020004', '200408070002', '200408090003', '200408120002', '200408240002', '200408240004', '200408250005', '200408260010', '200408280006', '200408300002', '200409100006', '200409120006', '200409250002', '200409280006', '200410030001', '200410040004', '200410070004', '200410100004', '200410160006', '200410250008', '200410310003', '200411050002', '200411060002', '200411070004', '200411080004', '200411090003', '200411150002', '200411250003', '200412030002', '200412060004', '200412090002', '200412100009']


In [39]:
print(len(G_2004_kernel))

55


Класс, посчитали число связных компонент! В прицнипе метрика не бесполезная - вроде как число не связных между собой групп террактов достаточно полезная информация.

# Своя метрика

Графы такой структуры (много маленьких полносвязных графов) можно закодировать в вектор, в каждой компоненте которого - размер связной компоненты, такой вектор хранит абсолютно всю структуру о графе(учитывая то, что мы знаем, как они устроены), и на таких векторах можно считать расстояния, например, сумму квадратов разности компонент, предварительно отсортировав компоненты и добив нулями до одинаковой длины. Такая метрика будет сильно скакать, если компоненты связности сильно отличаются по размеру, будет учитывать разность длин.

In [40]:
def my_vec(G):
    return np.array([len(cc) for cc in nx.connected_components(G)])

In [41]:
my_vec(G_2001)

array([2, 2, 4, 3, 4, 4, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 2, 3, 5, 2, 2, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 4, 2, 3, 5, 2, 2, 2, 7, 2, 2, 3, 2, 2, 3, 3, 2, 9, 2, 2, 3,
       2, 2, 2, 2, 3, 3, 4, 2, 2, 3, 3, 4, 4, 8, 3, 3, 3, 2, 3, 2, 2, 2,
       2, 3, 6, 2, 2, 2, 2, 6, 2, 5])

In [42]:
def my_metric(G_1, G_2):
    v1 = sorted(my_vec(G_1), reverse=True) # type: list
    v2 = sorted(my_vec(G_2), reverse=True)
    max_len = max(len(v1), len(v2))
    v1_np = np.zeros(max_len)
    v2_np = np.zeros(max_len)
    v1_np[:len(v1)] = v1
    v2_np[:len(v2)] = v2
    return np.sum((v1_np - v2_np) ** 2)

In [43]:
my_metric(G_2001, G_2004)

205.0

отнормируем

In [44]:
def my_normed_metric(G_1, G_2):
    v1 = sorted(my_vec(G_1), reverse=True) # type: list
    v2 = sorted(my_vec(G_2), reverse=True)
    max_len = max(len(v1), len(v2))
    v1_np = np.zeros(max_len)
    v2_np = np.zeros(max_len)
    v1_np[:len(v1)] = v1
    v2_np[:len(v2)] = v2
    return np.sum((v1_np - v2_np) ** 2) / np.sqrt(np.sum(v1_np ** 2)) / np.sqrt(np.sum(v2_np ** 2))

In [45]:
my_normed_metric(G_2001, G_2004)

0.2963015625337177

Очень близко к самому полезного из увиденного - нормированному редакционному расстоянию.